# Import Libraries

In [14]:
import pandas as pd
from itertools import zip_longest
from bs4 import BeautifulSoup
from selenium import webdriver

from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [15]:
# code to force show all column
pd.set_option('display.max_columns', None)

# max column width
pd.set_option('display.max_colwidth', 100)

# Load Dataset

In [16]:
df = pd.read_csv('../datasets/rumah123_hyperlink.csv')

# Scrap Web Per Pages

In [17]:
#init webdriver & test BeautifulSoup
driver = webdriver.Chrome()

url="https://www.rumah123.com/jual/rumah/?q=Balikpapan"
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html, "html.parser") #Menentukan parser HTML yang ingin kita gunakan.
print(soup.prettify()[:700]) # memberikan representasi visual dari pohon parse yang dibuat dari konten HTML mentah.

<html lang="id-ID">
 <head>
  <script async="" id="analytics-js" src="https://cdn.segment.com/analytics.js/v1/fpueXeNRJ8uBgCLuBDRDkZFAMBT0yLLb/analytics.min.js" type="text/javascript">
  </script>
  <script>
   partytown = {
                "debug": false,
                "lib": "/asset-core/script/partytown/",
                "logCalls": true,
                "logGetters": true,
                "logImageRequests": true,
                "logMainAccess": true,
                "logScriptExecution": true,
                "logSendBeaconRequests": true,
                "logStackTraces": true,
                "forward": ['dataLayer.push', 'fbq', '_hsq.push', 'ttq.track', 'ttq.page', 'ttq.load'],
 


In [18]:
# prepare variable to save the datas
id = []
judul = []
harga = []
lokasi = []
kamar_tidur = []
kamar_mandi = []
luas_tanah = []
luas_bangunan = []
carport = []
tipe_properti = []
sertifikat = []
daya_listrik = []

kamar_pembantu = []
dapur = []
ruang_makan = []
ruang_tamu = []
kondisi_perabotan = []

# Exterior
jumlah_lantai = []
pemandangan = []

# tentang properti
terjangkau_internet = []
lebar_jalan = []
sumber_air = []
hook = []
kondisi_properti = []
tipe_iklan = []
id_iklan = []

### Changed Logic
1. First initial labels of all possible data
2. Create a dictionary based on the labels
3. Create a Loop to scrap the data and store it in the dictionary
    - First, fill the dictionary with NaN for all key so that when we didn't found the data, it still in the same shape
    - Loop 1: To get the labels1 data (Kamar Tidur, Kamar Mandi, etc)
    - Loop 2: To get the labels2 data (Luas Bangunan, Luas Tanah, etc)

In [20]:
# clean dictionary
data1 = {}
data2 = {}
harga = []
judul = []
lokasi = []
harga = [np.nan]*1467
judul = [np.nan]*1467
lokasi = [np.nan]*1467
urls = [np.nan]*1467
img_hyperlinks = [np.nan]*1467

# define the labels
labels1 = ['Kamar Tidur', 'Kamar Mandi', 'Luas Tanah', 'Luas Bangunan', 'Carport', 'Tipe Properti', 'Sertifikat', 'Daya Listrik']
labels2 = ['Kamar Pembantu', 'Dapur', 'Ruang Makan', 'Ruang Tamu', 'Kondisi Perabotan', 'Jumlah Lantai', 'Pemandangan', 'Terjangkau Internet', 'Lebar Jalan', 'Sumber Air', 'Hook', 'Kondisi Properti', 'Tipe Iklan']

# create dictionaries to store the labels and their values
data1 = {label: [np.nan]*1467 for label in labels1}
data2 = {label: [np.nan]*1467 for label in labels2}

loop = 365
print("Progresss: ")

# traverse the hyperlink to get the data needed (test 10 data first)
for i in df['hyperlink'][365:375]:
    
    url = i
    urls[loop] = url
    img_hyperlinks[loop] = df['img_hyperlinks'][loop]
    
    url=i
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # generate random number between 1 and 3
    rand = np.random.randint(1, 17)

    # short pause if loop is a multiple of 5, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 5) == 0) and (loop != 0):
        pause = 25 - rand
        time.sleep(pause)
        # print(f'Long Pause for {pause} seconds...')
    
    # Long pause if loop is a multiple of 19, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 19) == 0) and (loop != 0):
        pause = 79 - rand
        print(f'Long Pause for {pause} seconds...')
        time.sleep(pause)
    
    # # Super long pause if loop is a multiple of 5, pause the execution for a specified amount of time (e.g., 60 seconds)
    # if ((loop % 98) == 0) and (loop != 0):
    #     pause = 820 - rand
    #     print(f'Super Long Pause for {pause} seconds...')
    #     time.sleep(pause)
    
    
    get_price_iklan = soup.find_all('div', class_="r123-listing-summary__price")
    # get the span inside the div
    for i in get_price_iklan:
        get_price_iklan = i.find('span')  # use find instead of find_all
        harga[loop] = get_price_iklan.text

    judul_elements = soup.find_all('h1', {"class":"r123-listing-summary__header-container-title"})
    if judul_elements:
        judul[loop] = judul_elements[0].get_text()

    lokasi_elements = soup.find_all('div', {"class":"r123-listing-summary__header-container-address"})
    if lokasi_elements:
        lokasi[loop] = lokasi_elements[0].get_text()
    
    # ----------------- #

    # first loop
    spec_items = soup.find_all('div', class_="listing-specification-v2__item")
    for item in spec_items:
        label = item.find('span', class_="listing-specification-v2__item-label").text
        value = item.find('span', class_="listing-specification-v2__item-value").text

        # if the label is in the first dictionary, insert the value at the index of the current loop
        if label in data1:
            data1[label][loop] = value

    # second loop
    interior_items = soup.find_all('div', class_="ui-molecule-accordion-r123")
    for item in interior_items:
        spec_items = item.find_all('div', class_="listing-specification-v2__item")
        for spec_item in spec_items:
            label = spec_item.find('span', class_="listing-specification-v2__item-label").text
            value = spec_item.find('span', class_="listing-specification-v2__item-value").text

            # if the label is in the second dictionary, insert the value at the index of the current loop
            if label in data2:
                data2[label][loop] = value
   
    #----------------- #
    loop += 1
    
    # print loop progress total number of link divided by hyperlinks
    print(f'Done: {loop} / {df["hyperlink"].size}')    

Progresss: 
Done: 366 / 1467
Done: 367 / 1467
Done: 368 / 1467
Done: 369 / 1467
Done: 370 / 1467
Done: 371 / 1467
Done: 372 / 1467
Done: 373 / 1467
Done: 374 / 1467
Done: 375 / 1467


In [22]:
# convert the dictionaries to dataframes
judull = pd.DataFrame(judul, columns=['Judul'])
hargaa = pd.DataFrame(harga, columns=['Harga'])
lokasia = pd.DataFrame(lokasi, columns=['Lokasi'])
hyperlink_df = pd.DataFrame(urls, columns=['Hyperlink'])
img_hyperlink_df = pd.DataFrame(img_hyperlinks, columns=['Img_Hyperlink'])

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# concatenate the dataframes
df_fix = pd.concat([judull, hargaa, lokasia, hyperlink_df, img_hyperlink_df, df1, df2], axis=1)
print(df_fix.shape)

# drop all rows where Judul is NaN
df_fix = df_fix.dropna(subset=['Judul'])

df_fix

(1467, 26)


,Judul,Harga,Lokasi,Hyperlink,Img_Hyperlink,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
365,RUMAH LUAS MEWAH DI TENGAH KOTA BALIKPAPAN,"Rp 1,5 Miliar","Telaga Sari, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8748565/,https://picture.rumah123.com/r123/720x420-crop/house/ho87/8748565/original/hos8748565-rumah-di-j...,5,3,137 m²,274 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
366,"BALIKPAPAN, RUMAH CANTIK TENGAH KOTA (Y255)","Rp 1,6 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8598225/,https://picture.rumah123.com/r123/720x420-crop/house/ho85/8598225/original/hos8598225-rumah-di-j...,3,2,210 m²,176 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,NaN,NaN,NaN,NaN,Semi Furnished,NaN,NaN,NaN,NaN,NaN,NaN,Bagus,Dijual
367,DUA RUMAH BERSEBELAHAN SATU SERTIFIKAT SIAP HUNI,"Rp 1,5 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8571144/,https://picture.rumah123.com/r123/720x420-crop/house/ho85/8571144/original/hos8571144-rumah-di-j...,5,4,193 m²,150 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,4400 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
368,Rumah Kosong di Balikpapan,Rp 795 Juta,"Sepinggan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8487441/,https://picture.rumah123.com/r123/720x420-crop/house/ho84/8487441/original/hos8487441-rumah-di-j...,3,1,150 m²,63 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,Unfurnished,NaN,NaN,NaN,NaN,NaN,NaN,Butuh Renovasi,Dijual
369,Rumah Mewah design minimalis siap huni Cluster Paris Balikpapan Baru,"Rp 7,5 Miliar","Balikpapan Baru, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8386551/,https://picture.rumah123.com/r123/720x420-crop/house/ho83/8386551/original/hos8386551-rumah-di-j...,NaN,NaN,300 m²,300 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,10000 Watt,NaN,NaN,NaN,NaN,Furnished,1,NaN,NaN,NaN,NaN,NaN,Bagus,Dijual
370,Ruman modern lokasi premium dan elite,"Rp 3,8 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos7171244/,https://picture.rumah123.com/r123/720x420-crop/house/ho71/7171244/original/hos7171244-rumah-di-j...,4,3,240 m²,245 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,NaN,1,NaN,NaN,NaN,Unfurnished,1,NaN,NaN,NaN,NaN,NaN,Bagus Sekali,Dijual
371,"Rumah cantik minimalis, modern, lokasi ciputra hadap boulevard","Rp 3,5 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos4742484/,https://picture.rumah123.com/r123/720x420-crop/house/ho47/4742484/original/hos4742484-rumah-di-j...,4,5,300 m²,300 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,NaN,NaN,NaN,NaN,NaN,Unfurnished,1,NaN,NaN,NaN,NaN,NaN,Bagus,Dijual
372,rmh cantik siap huni,"Rp 1,2 Miliar","Balikpapan Timur, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos4274768/,https://picture.rumah123.com/r123/720x420-crop/house/ho42/4274768/original/hos4274768-rumah-di-j...,3,2,152 m²,70 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,Semi Furnished,1,NaN,NaN,NaN,NaN,NaN,Bagus Sekali,Dijual
373,"Kost Baru + Rumah 2 lantai, 28 + 4 Kamar di Gunung Sari, Posisi Strategis",Rp 3 Miliar,"Balikpapan Tengah, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos2976033/,https://picture.rumah123.com/r123/720x420-crop/house/ho29/2976033/original/hos2976033-rumah-di-j...,32,NaN,525 m²,305 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,NaN,NaN,NaN,Tidak,Tidak,Furnished,NaN,NaN,Tidak,NaN,NaN,Tidak,Butuh Renovasi,Dijual
374,TURUN HARGA LAGI KANTOR/ RUMAH STRATEGIS HARGA MURAH,"Rp 4,5 Miliar","Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos1942180/,https://picture.rumah123.com/r123/720x420-crop/house/ho19/1942

In [ ]:
# to csv
# df_fix.to_csv('./datasets/rumah123_data_single_raw_fix_part_2.csv', index=True)

In [ ]:
# get numbers of index that hasnt been scrapped from 501 to 1000



Judul                    0
Harga                    0
Lokasi                   0
Kamar Tidur             15
Kamar Mandi             12
Luas Tanah               0
Luas Bangunan            0
Carport                394
Tipe Properti            0
Sertifikat               0
Daya Listrik            20
Kamar Pembantu         396
Dapur                  394
Ruang Makan            241
Ruang Tamu             241
Kondisi Perabotan      357
Jumlah Lantai          344
Pemandangan            383
Terjangkau Internet    241
Lebar Jalan            384
Sumber Air             376
Hook                   241
Kondisi Properti       321
Tipe Iklan               0
dtype: int64

In [ ]:
# full inner join on judul and Judul delete the rest that not match
df_fix_combine = pd.merge(df, df_fix, left_on='judul', right_on='Judul', how='inner')

# drop where Judul is NaN
df_fix_combine = df_fix_combine.dropna(subset=['Judul'])

In [ ]:
df_fix_combine.sample(10)

,judul,hyperlink,img_hyperlinks,Judul,Harga,Lokasi,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
294,RUMAH ETNIK DI BATU AMPAR DENGAN KONSEP CONTTAGE ASRI DAN NYAMAN,https://www.rumah123.com//properti/balikpapan/hos13884122/,https://picture.rumah123.com/r123/720x420-crop/house/ho13/13884122/original/hos13884122-rumah-di...,RUMAH ETNIK DI BATU AMPAR DENGAN KONSEP CONTTAGE ASRI DAN NYAMAN,"Rp 4,25 Miliar","Batu Ampar, Balikpapan",5,6,770 m²,350 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,4400 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
266,rumah model Belanda bisa tempat tinggal ada usaha kost,https://www.rumah123.com//properti/balikpapan/hos14630080/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14630080/original/hos14630080-rumah-di...,rumah model Belanda bisa tempat tinggal ada usaha kost,Rp 970 Juta,"Balikpapan Selatan, Balikpapan",5,3,120 m²,185 m²,2,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,NaN,NaN,Tidak,Tidak,Unfurnished,NaN,Pemukiman Warga,Tidak,3 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
63,TAMAN SARI BUKIT MUTIARA (WIKA),https://www.rumah123.com//properti/balikpapan/hos9907448/,https://picture.rumah123.com/r123/720x420-crop/house/ho99/9907448/original/hos9907448-rumah-di-j...,TAMAN SARI BUKIT MUTIARA (WIKA),"Rp 1,5 Miliar","Balikpapan Utara, Balikpapan",4,2,200 m²,200 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
257,RUMAH MINIMALIS 2 LANTAI SIAP HUNI DENGAN TAMBAHAN TANAH YANG CUKUP LUAS,https://www.rumah123.com//properti/balikpapan/hos14836211/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14836211/original/hos14836211-rumah-di...,RUMAH MINIMALIS 2 LANTAI SIAP HUNI DENGAN TAMBAHAN TANAH YANG CUKUP LUAS,"Rp 1,25 Miliar","Balikpapan Utara, Balikpapan",2,2,60 m²,47 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual
68,BALIKPAPAN BARU,https://www.rumah123.com//properti/balikpapan/hos9907443/,https://picture.rumah123.com/r123/720x420-crop/house/ho99/9907443/original/hos9907443-rumah-di-j...,BALIKPAPAN BARU,"Rp 2,5 Miliar","Damai, Balikpapan",4,2,200 m²,275 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
45,BALIKPAPAN PERUMAHAN REGENCY,https://www.rumah123.com//properti/balikpapan/hos14863499/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14863499/original/hos14863499-rumah-di...,BALIKPAPAN PERUMAHAN REGENCY,"Rp 1,75 Miliar","Balikpapan Selatan, Balikpapan",3,2,152 m²,115 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
155,Hunian modern dekat rumah sakit umum Kanujoso,https://www.rumah123.com//properti/balikpapan/hos16150027/,https://picture.rumah123.com/r123/720x420-crop/house/ho16/16150027/original/hos16150027-rumah-di...,Hunian modern dekat rumah sakit umum Kanujoso,"Rp 1,01 Miliar","Balikpapan Utara, Balikpapan",4,2,120 m²,72 m²,2,Rumah,HGB - Hak Guna Bangunan,2200 Watt,NaN,NaN,Ya,Ya,NaN,NaN,Pemukiman Warga,Ya,2 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
210,HARGA SPESIAL RUMAH KANTOR 2LT DI TENGAH KOTA,https://www.rumah123.com//properti/balikpapan/hos15688469/,https://picture.rumah123.com/r123/720x420-crop/house/ho15/15688469/original/hos15688469-rumah-di...,HARGA SPESIAL RUMAH KANTOR 2LT DI TENGAH KOTA,"Rp 2,1 Miliar","Balikpapan Tengah, Balikpapan",8,6,429 m²,429 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,5500 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
376,Bu Rumah Komplek Full Furnish di Jl Wonorejo Balikpapan Utara,https://www.rumah123.com//properti/balikpapan/hos16140392/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1476447/2024-01-23-07-02-24-03562...,Bu Rumah Komplek Full Furnish di Jl Wonorejo Balikp

In [ ]:
# check null values
df_fix_combine.isnull().sum()

judul                    0
hyperlink                0
img_hyperlinks           0
Judul                    0
Harga                    0
Lokasi                   0
Kamar Tidur             15
Kamar Mandi             12
Luas Tanah               0
Luas Bangunan            0
Carport                546
Tipe Properti            0
Sertifikat               0
Daya Listrik            51
Kamar Pembantu         533
Dapur                  533
Ruang Makan            330
Ruang Tamu             330
Kondisi Perabotan      477
Jumlah Lantai          458
Pemandangan            533
Terjangkau Internet    330
Lebar Jalan            534
Sumber Air             527
Hook                   330
Kondisi Properti       435
Tipe Iklan               0
dtype: int64

In [ ]:
# to csv
# df_fix_combine.to_csv('./datasets/rumah123_fix_data_fixxx_part_2.csv')